In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import re
import keras
import random
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.applications import MobileNetV2,DenseNet121, MobileNet, ResNet152V2, InceptionV3,Xception,InceptionResNetV2,VGG19, ResNet50, NASNetLarge, DenseNet201,VGG16, DenseNet169, ResNet152V2


In [ ]:
import numpy as np
import h5py

hf = h5py.File('../input/breastnew4k/Train224x224xnew.h5', 'r')
hf.keys()

x = np.array(hf.get('x'))
y = np.array(hf.get('y'))


labels = np.load('../input/breastcancer225x500/Cancerlabels.npy')

hf.close()

del hf

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(y)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x , y , test_size = 0.2 , stratify = y , random_state = 0)

In [ ]:
del x,y#,data

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
with strategy.scope():
    pre_trained_model = DenseNet121(input_shape=(224,224,3),include_top=False, weights="imagenet")
    for layer in pre_trained_model.layers[:19]:
        layer.trainable = False
    

    model = Sequential([
        pre_trained_model,
        MaxPool2D((2,2) , strides = 2),
        Flatten(),
        Dense(4 , activation='softmax')])
    model.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

In [ ]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 3, verbose=1,factor=0.3, min_lr=0.000001)

In [ ]:
history = model.fit(x_train,y_train, batch_size = 64 , epochs =100  , validation_data = (x_test, y_test),callbacks = [learning_rate_reduction])

In [ ]:
print("Loss of the model is - " , model.evaluate(x_test,y_test)[0])
print("Accuracy of the model is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

In [ ]:
predictions = model.predict_classes(x_test)
predictions[:4]

In [ ]:
y_test_inv = label_binarizer.inverse_transform(y_test)

In [ ]:
print(classification_report(y_test_inv, predictions, target_names = labels))

In [ ]:
cm = confusion_matrix(y_test_inv,predictions)

In [ ]:
cm = pd.DataFrame(cm , index = labels , columns = labels)
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='' , xticklabels = labels , yticklabels = labels)

In [ ]:
model.save('BreastCancer224_mobilenetv2121.h5')

In [ ]:
model = tf.keras.models.load_model('../input/modelbreatcancer224/ModelBreastCancer224.h5')

model.summary()

In [ ]:
path = '/tmp/simple_keras_model'
tf.keras.experimental.export_saved_model(model, path)

In [ ]:
tf.keras.experimental.export_saved_model(model, 'model.h5')

In [ ]:
def predict(path):
#path ='../input/asdfghjkl/begin.png'
    img = cv2.imread(path)
    plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
    #plt.show()
    
    img_size=225
    img = cv2.resize(img,(img_size,img_size),3)
    img = np.array(img) / 255
    img = np.array(img)
    img = img.reshape(-1, img_size, img_size, 3)
    # Generate predictions for samples
    predictions = model.predict(img)
    #print(predictions)

    # Generate arg maxes for predictions
    classes = np.argmax(predictions, axis = 1)
    for i in classes:
        if(i==0):print('Dự đoán :','Benign')
        elif(i==1):print('Dự đoán :','InSitu')
        elif(i==2):print('Dự đoán :','Invasive')
        else:print('Dự đoán :','Normal')
    print ('Độ chính xác:',np.max(predictions)*100,'%')

In [ ]:
predict(r'../input/testdata/test37.tif')

In [ ]:
listFile = os.walk(r'../input/testdata')
for path, subdirs, files in listFile:
    for name in files:
        print(name)
        predict(os.path.join(path, name))
